# Manually Scaling Image Pixel Data

## 1. Loading Image

In [1]:
# load and show an image with pillow
from PIL import Image
# load the image
image = Image.open('full_grain_1.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()

JPEG
RGB
(2742, 2250)


## 2. Normalize Pixel Values
* It is good practice to normalize the pixel values so that each pixel value has a value between 0 and 1.

In [2]:
# example of pixel normalization
from numpy import asarray
from PIL import Image
# load image
image = Image.open('full_grain_1.jpg')
pixels = asarray(image)
# confirm pixel range is 0-255
print('Data Types: %s' %pixels.dtype)
print('Min: %.3f, Max: %.3f' %(pixels.min(),pixels.max()))
# convert from integers to floats
pixels = pixels.astype('float32')
# normalize to the range 0-1
pixels/=255.0
# confirm the normalization
print('Min: %.3f, Max: %.3f' %(pixels.min(),pixels.max()))

Data Types: uint8
Min: 10.000, Max: 255.000
Min: 0.039, Max: 1.000


## 3. Centering Pixel Values

### 3.1 Global Centering
* Calculating and subtracting the mean pixel value across color channels

In [4]:
# example of global centering (subtract mean)
from numpy import asarray
from PIL import Image
# load image
image = Image.open('full_grain_1.jpg')
pixels = asarray(image)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate global mean
mean = pixels.mean()
print('Mean: %.3f' %mean)
print('Min: %.3f, Max: %.3f' %(pixels.min(),pixels.max()))
# global centering of pixels
pixels = pixels - mean
# confirm it had the desired effect
mean = pixels.mean()
print('Mean: %.3f' %mean)
print('Min: %.3f, Max: %.3f' %(pixels.min(),pixels.max()))

Mean: 126.873
Min: 10.000, Max: 255.000
Mean: -0.000
Min: -116.873, Max: 128.127


### 3.2 Local Centering
* Calculating and subtracting the mean pixel value per color channel.

In [5]:
# example of per-channel centering(subtract mean)
from numpy import asarray
from PIL import Image
# load image
image = Image.open('full_grain_1.jpg')
pixels = asarray(image)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate per-channel means and standard deviations
means = pixels.mean(axis=(0,1),dtype='float64')
print('Means: %s' %means)
print('Mins: %s, Maxs: %s' %(pixels.min(axis=(0,1)),pixels.max(axis=(0,1))))
# per-channel centering of pixels
pixels -= means
# confirm it had the desired effect
means = pixels.mean(axis=(0,1),dtype='float64')
print('Means: %s' %means)
print('Mins: %s, Maxs: %s' %(pixels.min(axis=(0,1)),pixels.max(axis=(0,1))))


Means: [ 87.85478758  94.09913542 198.66435708]
Mins: [11. 10. 50.], Maxs: [255. 255. 255.]
Means: [ 3.56752334e-07 -4.45384173e-07  3.47605516e-07]
Mins: [ -76.85479  -84.09914 -148.66435], Maxs: [167.14522  160.90086   56.335644]


## 4. Standardizing Pixel Values
* The result is a standard Gaussian of Pixel values with a mean of 0.0 and a standard deviation of 1.0

### 4.1 Global Standardization
* It calculates the mean and standard deviations across all color channels in the loaded image,then uses these values to standardize the pixel values.

In [6]:
# example of global pixel standardization
from numpy import asarray
from PIL import Image
# load image
image = Image.open('full_grain_1.jpg')
pixels = asarray(image)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate global mean and standard deviation
mean,std = pixels.mean(),pixels.std()
print('Mean: %.3f,Standard Deviation: %.3f' %(mean,std))
# global standardization of pixels
pixels = (pixels - mean) / std
# confirm it had desired effect
mean,std = pixels.mean(),pixels.std()
print('Mean: %.3f,Standard Deviation: %.3f' %(mean,std))


Mean: 126.873,Standard Deviation: 63.042
Mean: -0.000,Standard Deviation: 1.000


### 4.2 Positive Global Standardization
* A popular wayb of achieving this is to clip the standardized pixel values to the range [-1,1] and then rescale the values from [-1,1] to [0,1].

In [8]:
# example of global pixel standardization shifted to positive domain
from numpy import asarray
from numpy import clip
from PIL import Image
# load image
image = Image.open('full_grain_1.jpg')
pixels = asarray(image)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate global mean and standard deviation
mean,std = pixels.mean(),pixels.std()
print('Mean: %.3f, Standard Deviation: %.3f' %(mean,std))
# global standardization of pixels
pixels = (pixels - mean) / std
# clip pixel values to [-1,1]
pixels = clip(pixels,-1.0,1.0)
# shift from [-1,1] to [0,1] with 0.5 mean
pixels = (pixels + 1.0) / 2.0
# confirm it had the desired effect
mean,std = pixels.mean(),pixels.std()
print('Mean: %.3f, Standard Deviation: %.3f' %(mean,std))
print('Min: %.3f, Max: %.3f' %(pixels.min(),pixels.max()))

Mean: 126.873, Standard Deviation: 63.042
Mean: 0.461, Standard Deviation: 0.417
Min: 0.000, Max: 1.000


### 4.3 Local Standardization
* It calculates the mean and standard deviation of the loaded image per-channel,then uses these statistics to standardize the pixels separately in each channel. 

In [9]:
# example of per-channel pixel standardization
from numpy import asarray
from PIL import Image
# load image
image = Image.open('full_grain_1.jpg')
pixels = asarray(image)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate per-channel means and standard deviations
means = pixels.mean(axis=(0,1),dtype='float64')
stds = pixels.std(axis=(0,1),dtype='float64')
print('Means: %s, Stds: %s' %(means,stds))
# per-channel standardization of pixels
pixels = (pixels - means) / stds
# confirm it had the desired effect
means = pixels.mean(axis=(0,1),dtype='float64')
stds = pixels.std(axis=(0,1),dtype='float64')
print('Means: %s, Stds: %s' %(means,stds))

Means: [ 87.85478758  94.09913542 198.66435708], Stds: [43.59767413 42.56466188 21.44376707]
Means: [ 8.63717132e-12 -2.12247090e-13 -1.11486289e-13], Stds: [1. 1. 1.]
